## Likelihood comparison
Compare the likelihood and its components between the four candidate analyses (with or without the proposed extensions).

### Running over Colab
Uncomment the commands in the following cell to install in Colab.
Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment and then proceed to the following cells.

### Generating documentation
Running the final cell will take several minutes and will generate the corresponding PDF documentation.
This can be downloaded from the Files section of the sidebar.

In [ ]:
# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.9 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch main
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
from emutools.calibration import get_like_components, plot_like_components_by_analysis
from emutools.tex import StandardTexDoc, add_image_to_doc
from inputs.constants import SUPPLEMENT_PATH, PRIMARY_ANALYSIS
from aust_covid.extra_text import add_likelihood_blurb_to_tex

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'likelihood_comparison', 'Analysis comparison', 'austcovid')

In [ ]:
requests = {
    'loglikelihood': (-28.0, -12.0, 'total likelihood'),
    'll_adult_seropos_prop': (-3.0, 2.0, 'seroprevalence contribution'),
    'll_deaths_ma': (-10.0, -3.0, 'deaths contribution'),
    'll_notifications_ma': (-17.0, -10.0, 'cases contribution'),
}
like_outputs = get_like_components(requests.keys())
like_comparison_fig = plot_like_components_by_analysis(like_outputs, 'kdeplot', plot_requests=requests, alpha=0.1, linewidth=1.5);
caption = 'Comparison of the kernel density distribution of the final likelihood from calibration algorithm, ' \
    'with the contributions to the final likelihood of the three targets from which it was constructed.'
add_image_to_doc(like_comparison_fig, 'like_comparison', 'svg', 'Likelihood comparison, kernel densities.', app_doc, 'Analysis comparison', caption=caption)

Likelihood contribution statements made in manuscript.

In [ ]:
like_outputs['ll_adult_seropos_prop'].median()

In [ ]:
like_outputs['ll_notifications_ma'].median()

In [ ]:
like_outputs['ll_deaths_ma'][PRIMARY_ANALYSIS].median()

In [ ]:
violin_comparison_fig = plot_like_components_by_analysis(like_outputs, 'violinplot', requests);
add_image_to_doc(violin_comparison_fig, 'violin_comparison', 'svg', 'Likelihood comparison, violin plots.', app_doc, 'Analysis comparison', caption=caption)
add_likelihood_blurb_to_tex(app_doc, cross_ref=False, fig_ref=False)
app_doc.write_doc()

In [ ]:
plot_like_components_by_analysis(like_outputs, 'histplot', plot_requests=requests);

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y update
    ! apt-get -y install inkscape texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex -shell-escape likelihood_comparison
    ! pdflatex -shell-escape likelihood_comparison
    ! pdflatex -shell-escape likelihood_comparison